## Quantum Reservoir Computing for Chaotic Dynamics
FYP Masters project by Raihaan Usman

In [4]:
from backend.solver import Lorentz, MFE
from backend import CRCM, QRCM

import numpy as np
import wandb
import os;  os.environ["OMP_NUM_THREADS"] = '32' # Imposes cores

#### Configuration for Lorentz system

In [8]:
# Set up WandB
wandb.login()

sweep_config = {
    'method'    : 'bayes',
    'name'      : 'QRCM sweep',
    'metric'    : {
        'name'  : 'MSE',
        'goal'  : 'minimize'
    },
    'parameters'    : {
        'qubits'    : {'values': [2, 4, 6]},
        'epsilon'   : {'min': 0.0, 'max': 1.0},
        'tikhonov'  : {'min': 1e-10, 'max': 1e-2},
        'N_train'   : {'values': [50, 100, 300, 500, 1000, 2000]},
        'N_test'    : {'values': [50, 100, 300, 500, 1000, 2000]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="Quantum Turbulence Learning", entity="raihaan123")

# Data generation parameters
dim             = 3
upsample        = 1                     # To increase the dt of the ESN wrt the numerical integrator
dt              = 0.005 * upsample      # Time step
params          = [8/3, 28, 10]         # Parameters for the Lorenz system

Create sweep with ID: oi98d2qa
Sweep URL: https://wandb.ai/raihaan123/Quantum%20Turbulence%20Learning/sweeps/oi98d2qa


#### CRCM for Lorenz system

In [9]:
# # Initialise the ESN
# crcm = CRCM(dim=dim,
#             N_units=200,
#             connectivity=3,
#             seed=0)

# # Train the ESN with the training data
# crcm.train(data)

#### QRCM for Lorenz system

In [10]:
def train():
    with wandb.init() as run:
        config = wandb.config

        # Define N for washout, training, validation and testing
        N_washout       = 50
        N_train         = config['N_train']
        N_test          = config['N_test']
        N_sets          = [N_washout, N_train, N_test]

        # Instantiate the solver object
        lor3 = Lorentz(params, dt, N_sets)

        # Instantiate the QRCM object
        qrcm = QRCM(solver  = lor3,
                    qubits  = config['qubits'],
                    eps     = config['epsilon'],
                    tik     = config['tikhonov'])

        # Train the QRCM with the training data
        qrcm.train(override=True)

        # Log data to WandB - can generate plots from this
        wandb.log({'Error Time Series': qrcm.err_ts})
        wandb.log({'MSE': qrcm.MSE})
        wandb.log({'Time': qrcm.time})              # Log the time taken to train the model


# Number of runs to execute
wandb.agent(sweep_id, function=train, count=100)

wandb: Agent Starting Run: m61be58n with config:
wandb: 	N_test: 500
wandb: 	N_train: 300
wandb: 	epsilon: 0.5291352435787909
wandb: 	qubits: 6
wandb: 	tikhonov: 0.0009797388107234625
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Testing: 100%|██████████| 499/499 [00:55<00:00,  8.93it/s]


Run m61be58n errored: AttributeError("'QRCM' object has no attribute 'samp'")
wandb: ERROR Run m61be58n errored: AttributeError("'QRCM' object has no attribute 'samp'")
wandb: Agent Starting Run: p34tk0ov with config:
wandb: 	N_test: 2000
wandb: 	N_train: 2000
wandb: 	epsilon: 0.66310661443464
wandb: 	qubits: 6
wandb: 	tikhonov: 0.006329111957502426
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Training:  64%|██████▍   | 1276/1999 [02:42<01:26,  8.38it/s]